In [2]:
import os
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers import GlobalAveragePooling2D, Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [3]:
# dimensi data
img_width = 50
img_height = 50

# channels
channels = 3

# epoch
num_epoch = 20

# learning rate
learn_rate = 1e-3

# directory train dan validation dataset
train_data_dir = 'data/train_bike_car_person'
validation_data_dir = 'data/val_bike_car_person'
test_data_dir = 'data/test_bike_car_person'

In [4]:
datagenerate = ImageDataGenerator(rescale = 1./255)

# melakukan transformasi pada data train
train_datagenerate = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True)

# pengambilan data set untuk train dan validasi
train_generator = train_datagenerate.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = 32,
    class_mode ='categorical')

validation_generator = datagenerate.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    batch_size = 32,
    class_mode = 'categorical')

test_generator = datagenerate.flow_from_directory(
    test_data_dir,
    target_size = (img_width, img_height),
    class_mode = None)

Found 1200 images belonging to 4 classes.
Found 276 images belonging to 4 classes.
Found 413 images belonging to 1 classes.


In [5]:
def create_cnn_model():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation = 'relu', padding='same', input_shape=(img_width,img_height,channels)))
    #model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #model.add(Conv2D(128, (3, 3), activation = 'relu', padding='same', strides = 2))
    #model.add(Dropout(0.2))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(32, (2, 2), activation = 'relu'))
    
    #model.add(Conv2D(32, (3, 3), activation = 'relu', padding='same', strides = 2))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #model.add(GlobalAveragePooling2D())
    
    model.add(Flatten())
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4))
    model.add(Activation('softmax'))

    
    model.summary()
    
    return model

In [6]:
model = create_cnn_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 50, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 64)          36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 32)          8224      
__________

In [6]:
# save the model after every epoch
checkpoint = ModelCheckpoint('best_model_improved.h5',
                            monitor = 'val_loss',
                            save_best_only = True,
                            mode = 'auto')

In [7]:
# konfigurasi model
model.compile(loss = 'categorical_crossentropy',
             optimizer = Adam(lr = learn_rate),
             metrics = ['accuracy'])

In [8]:
# Training
nb_train_samples = 1200
nb_validation_samples = 276
nb_test_samples = 413

In [9]:
# fit the model

model_details = model.fit_generator(
                                    train_generator,
                                    steps_per_epoch = nb_train_samples,
                                    nb_epoch = num_epoch,
                                    validation_data = validation_generator,
                                    validation_steps = nb_validation_samples,
                                    callbacks = [checkpoint])

C:\Users\Someone\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if __name__ == '__main__':
C:\Users\Someone\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_steps=276, epochs=10, callbacks=[<keras.ca..., validation_data=<keras.pre..., steps_per_epoch=1200)`
  if __name__ == '__main__':


Epoch 1/10
1200/1200 [==============================] - 64s - loss: 0.9347 - acc: 0.6190 - val_loss: 0.9099 - val_acc: 0.6231
Epoch 2/10
1200/1200 [==============================] - 55s - loss: 0.6075 - acc: 0.7723 - val_loss: 0.8852 - val_acc: 0.6460
Epoch 3/10
1200/1200 [==============================] - 56s - loss: 0.4284 - acc: 0.8371 - val_loss: 0.9669 - val_acc: 0.6822
Epoch 4/10
1200/1200 [==============================] - 57s - loss: 0.3009 - acc: 0.8885 - val_loss: 1.1168 - val_acc: 0.6816
Epoch 5/10
1200/1200 [==============================] - 59s - loss: 0.2300 - acc: 0.9178 - val_loss: 1.3126 - val_acc: 0.6736
Epoch 6/10
1200/1200 [==============================] - 58s - loss: 0.1704 - acc: 0.9418 - val_loss: 1.9462 - val_acc: 0.5980
Epoch 7/10
1200/1200 [==============================] - 60s - loss: 0.1439 - acc: 0.9518 - val_loss: 2.6808 - val_acc: 0.5974
Epoch 8/10
1200/1200 [==============================] - 61s - loss: 0.1129 - acc: 0.9626 - val_loss: 1.8954 - val_acc:

In [10]:
model.evaluate_generator(validation_generator, nb_validation_samples)

[0.88979858727079164, 0.643767704960307]

In [9]:
model.load_weights('best_model_improved.h5')

In [11]:
# prediksi dari data test

#model.predict(test_generator)
#model.predict_classes(test_generator)

# calculate predictions
predictions = model.predict_generator(test_generator, steps=nb_test_samples)

rounded = predictions.argmax(axis=-1)
print(rounded)

[3 0 3 ..., 0 0 1]


In [12]:
np.savetxt(fname="predict.txt", X=rounded)